# **PRÉ-PROCESSAMENTO**

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

In [4]:
df1 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_1s2022.csv", delimiter=';')
df2 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_2s2022.csv", delimiter=';')
df3 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_2s2021.csv", delimiter=';')
df4 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_1s2023.csv", delimiter=';')

In [5]:
frames = [df1, df2, df3, df4]
df = pd.concat(frames)

In [4]:
df.to_csv('zppoutput.csv', sep=';', encoding='utf-8', index = False)

In [26]:
df.head()

,ID,Order Number,Posting date,SKU,Quantity,Milestone,Entered by,Cost ctr
0,1820587547445640OO9417L94170359MARIANAAN,1820587547,03/01/2022,0OO9417L94170359,60,PRA,MARIANAAN,132-45565
1,1820871421445640AX4121S82948759CINTHIABDAR,1820871421,03/01/2022,0AX4121S82948759,37,LIN,CINTHIABDAR,132-45565
2,1821294046445640JK2007BJ16354GICELES,1821294046,03/01/2022,0JK2007BJ16354,69,EMB,GICELES,132-45565
3,1821294046445640JK2007BJ16354ALIDIA,1821294046,03/01/2022,0JK2007BJ16354,69,LIN,ALIDIA,132-45565
4,1821404980445640RB3663L004/7160MARIANAAN,1821404980,03/01/2022,0RB3663L004/7160,319,PRA,MARIANAAN,132-45565


## **Transformando o dataframe para exibir as variáveis de interesse**

In [6]:
df2 = pd.DataFrame.copy(df)

In [7]:
df2.drop('ID', axis=1, inplace=True)
df2.drop('Milestone', axis=1, inplace=True)
df2.drop('SKU', axis=1, inplace=True)
df2.drop('Order Number', axis=1, inplace=True)
df2.head()

,Posting date,Quantity,Entered by,Cost ctr
0,03/01/2022,60,MARIANAAN,132-45565
1,03/01/2022,37,CINTHIABDAR,132-45565
2,03/01/2022,69,GICELES,132-45565
3,03/01/2022,69,ALIDIA,132-45565
4,03/01/2022,319,MARIANAAN,132-45565


In [8]:
df3 = pd.DataFrame({'QTY': df2.groupby(by=['Cost ctr', 'Posting date'])['Quantity'].sum(),
                    'Employees QTY': df2.groupby(['Cost ctr', 'Posting date']).size()}).reset_index()
df3

,Cost ctr,Posting date,QTY,Employees QTY
0,132-07265,01/02/2022,1160,11
1,132-07265,01/02/2023,3330,13
2,132-07265,01/03/2023,1650,12
3,132-07265,01/04/2022,1320,18
4,132-07265,01/04/2023,1490,6
...,...,...,...,...
3608,132-45565,31/07/2021,43740,349
3609,132-45565,31/08/2021,76291,603
3610,132-45565,31/08/2022,66944,808
3611,132-45565,31/10/2022,58424,528


## **Transformando as variáveis categóricas nominais em variáveis categóricas ordinais**

In [9]:
df3['Cost ctr'].replace({'132-07265':0, '132-07565': 1, '132-10365':2, '132-31265': 3, '132-31865': 4, '132-45565': 5}, inplace=True)
df3.head()

,Cost ctr,Posting date,QTY,Employees QTY
0,0,01/02/2022,1160,11
1,0,01/02/2023,3330,13
2,0,01/03/2023,1650,12
3,0,01/04/2022,1320,18
4,0,01/04/2023,1490,6


In [13]:
df3['Posting date'] = pd.to_datetime(df3['Posting date'])
df3.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_10204\3786828873.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df3['Posting date'] = pd.to_datetime(df3['Posting date'])


,Cost ctr,Posting date,QTY,Employees QTY
0,0,2022-01-02,1160,11
1,0,2023-01-02,3330,13
2,0,2023-01-03,1650,12
3,0,2022-01-04,1320,18
4,0,2023-01-04,1490,6


In [14]:
df3.dtypes

Cost ctr                  int64
Posting date     datetime64[ns]
QTY                       int64
Employees QTY             int64
dtype: object

## **LEGENDA**

Cost ctr = centro de custo (setor da linha de produção)

Posting date = data de produção

QTY = quantidade de peças produzidas

Employees QTY = quantidade de funcionários

## **ATRIBUTOS PREVISORES E ALVO**

In [16]:
previsores = df3.iloc[:, 0:3].values
previsores

array([[0, Timestamp('2022-01-02 00:00:00'), 1160],
       [0, Timestamp('2023-01-02 00:00:00'), 3330],
       [0, Timestamp('2023-01-03 00:00:00'), 1650],
       ...,
       [5, Timestamp('2022-08-31 00:00:00'), 66944],
       [5, Timestamp('2022-10-31 00:00:00'), 58424],
       [5, Timestamp('2022-12-31 00:00:00'), 300]], dtype=object)

In [17]:
previsores.shape

(3613, 3)

In [19]:
alvo = df3.iloc[:, 3].values
alvo

array([ 11,  13,  12, ..., 808, 528,   1], dtype=int64)

In [20]:
alvo.shape

(3613,)

## **Análise das escalas dos atributos (Escalonamento)**

In [21]:
df3.describe()

,Cost ctr,QTY,Employees QTY
count,3613.000000,3613.000000,3613.000000
mean,2.603930,27463.464157,205.261279
std,1.670888,25948.141979,226.514060
min,0.000000,50.000000,1.000000
25%,1.000000,5273.000000,36.000000
50%,3.000000,17269.000000,107.000000
75%,4.000000,46587.000000,305.000000
max,5.000000,103292.000000,1039.000000


Padronização (utiliza a média e o desvio padrão como referência).

Normalização (utiliza os valores máximo e mínimo como referência).

In [ ]:
previsores_esc = StandardScaler().fit_transform(previsores)
previsores_esc